<h2>Task 1</h2>
<br>
<font color=blue><b>Calculate (manually *and* by programming in python), the odds for a given probability in the example dataset.</b>
<br><br>We have done this task with boht churn and titanic dataset

In [166]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data = pd.read_excel("churn.xlsx")

data.head()

data['Churn'].astype(str)
data.dtypes
#data.where(data['Chturn'].astype(str) != "NaN")
data.dropna(inplace=True)

overview = {
    "<=16": data[data['Age']<=16],
    "17-29": data[(data['Age']>=17) & (data['Age']<=29)],
    "30-41": data[(data['Age']>=30) & (data['Age']<=41)],
    "42-53": data[(data['Age']>=42) & (data['Age']<=53)],
    "54-66": data[(data['Age']>=54) & (data['Age']<=66)],
     ">=67": data[data['Age']>=67]
}

def odds(p):
    return p/(1-p)

#(overview["17-29"]['Churn']!='churn').sum()

for k in overview: 
    probability = np.round((overview[k]['Churn'] != 'churn').sum()/overview[k].shape[0],3)
    if probability.astype(str) == str('nan'):
        probability = 0
    print("The probability for bucket ",k," is ",probability)
    print("The odds for bucket ",k," are ",np.round(odds(probability),3),'\n')

The probability for bucket  <=16  is  0
The odds for bucket  <=16  are  0.0 

The probability for bucket  17-29  is  0.814
The odds for bucket  17-29  are  4.376 

The probability for bucket  30-41  is  0.617
The odds for bucket  30-41  are  1.611 

The probability for bucket  42-53  is  0.614
The odds for bucket  42-53  are  1.591 

The probability for bucket  54-66  is  0.547
The odds for bucket  54-66  are  1.208 

The probability for bucket  >=67  is  0.545
The odds for bucket  >=67  are  1.198 



In [165]:
df_train = pd.read_csv("titanic_train.csv")

In [4]:
df_train.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
1,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
2,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


In [21]:
#Calculating Probability of Survived:
prb_srv = df_train[df_train['Survived']==1].shape[0]/df_train.shape[0]
prb_die = 1-prb_srv
print('Probability of survival is',round(prb_srv,2))
print('Probability of death is',round(prb_die,2))

Probability of survival is 0.4
Probability of death is 0.6


In [19]:
#odds_srv = prob_surv / prob_die
odd_srv = prb_srv/prb_die
odd_die = prb_die/prb_srv
print('Odds of survival are',round(odd_srv,2))
print('Odds of death are',round(odd_die,2))

Odds of survival are 0.67
Odds of death are 1.48


<h2>Task 2</h2>
<font color=blue><br><b>Run the python code, change the models to include more or less dependent variables and test if the score improves or not.

<font color=blue>We have not used the original code provided in labs. Instead we have reimported the data set. In the the code below, you will see following steps:
<br>1- Importing Data Set
<br>2- Label encoding using pd.get_dummies
<br>3- Using only "Age" as independent variable and seeing the "Pseudo R-Squared" using Logit function
<br>4- Using all train columns as independent variable and seeing the "Pseudo R-Squared" using Logit function
<br>5- Assessing the difference and improvement, Removing columns that have P value less than 0.05
<br>6- Building a Logistic Regression model on selected columns and printing the accuracy score

<h2>Solution:

In [120]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings("ignore")

<h4>Using example given in Workshop assignment:

In [81]:
df = pd.read_excel('churn.xlsx')
df_orig = df.copy()
df.head()

,Gender,Age,Payment Method,Churn,LastTransaction
0,male,64,credit card,loyal,98
1,male,35,cheque,churn,118
2,female,25,credit card,loyal,107
3,female,39,credit card,NaN,177
4,male,39,credit card,loyal,90


In [82]:
df_new = pd.DataFrame(pd.get_dummies(df[['Gender','Payment Method','Churn']]))
df = pd.concat([df, df_new], axis=1, sort=False)
df.head()

,Gender,Age,Payment Method,Churn,LastTransaction,Gender_female,Gender_male,Payment Method_cash,Payment Method_cheque,Payment Method_credit card,Churn_churn,Churn_loyal
0,male,64,credit card,loyal,98,0,1,0,0,1,0,1
1,male,35,cheque,churn,118,0,1,0,1,0,1,0
2,female,25,credit card,loyal,107,1,0,0,0,1,0,1
3,female,39,credit card,NaN,177,1,0,0,0,1,0,0
4,male,39,credit card,loyal,90,0,1,0,0,1,0,1


In [87]:
#Dropping unnecessary columns
df =df.drop(columns=['Gender','Payment Method','Churn','Churn_loyal','Gender_female'],axis=0, errors = 'ignore')
df.head()

,Age,LastTransaction,Gender_male,Payment Method_cash,Payment Method_cheque,Payment Method_credit card,Churn_churn
0,64,98,1,0,0,1,0
1,35,118,1,0,1,0,1
2,25,107,0,0,0,1,0
3,39,177,0,0,0,1,0
4,39,90,1,0,0,1,0


In [88]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
def scale_it(df_X):
    df_X = pd.DataFrame(scaler.fit_transform(df_X),columns = df_X.columns.values)
    return df_X

In [89]:
df = scale_it(df)
df.head()

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,Age,LastTransaction,Gender_male,Payment Method_cash,Payment Method_cheque,Payment Method_credit card,Churn_churn
0,0.635135,0.436937,1.0,0.0,0.0,1.0,0.0
1,0.243243,0.527027,1.0,0.0,1.0,0.0,1.0
2,0.108108,0.477477,0.0,0.0,0.0,1.0,0.0
3,0.297297,0.792793,0.0,0.0,0.0,1.0,0.0
4,0.297297,0.400901,1.0,0.0,0.0,1.0,0.0


In [90]:
df.columns

Index(['Age', 'LastTransaction', 'Gender_male', 'Payment Method_cash',
       'Payment Method_cheque', 'Payment Method_credit card', 'Churn_churn'],
      dtype='object')

In [91]:
col_dict = {
    'train_cols' : ['Age', 'LastTransaction', 'Gender_male', 'Payment Method_cash',
       'Payment Method_cheque', 'Payment Method_credit card']
    ,'pred_cols' : ['Churn_churn']
    }

<font color= blue>Using only Age as predictor</font>

In [103]:
from statsmodels.api import Logit

Logit(df[col_dict['pred_cols']],df['Age']).fit().summary()

Optimization terminated successfully.
         Current function value: 0.670667
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            Churn_churn   No. Observations:                  996
Model:                          Logit   Df Residuals:                      995
Method:                           MLE   Df Model:                            0
Date:                Wed, 10 Jul 2019   Pseudo R-squ.:                -0.06569
Time:                        10:09:43   Log-Likelihood:                -667.98
converged:                       True   LL-Null:                       -626.81
                                        LLR p-value:                       nan
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Age           -0.9402      0.144     -6.528      0.000      -1.222      -0.658
==============================================================================
"""

<font color = blue>Let's add more columns now and see if Pseudo R-squ increases. We are also interested in removing columns that have P value less than 0.05.

In [104]:
Logit(df[col_dict['pred_cols']],df[col_dict['train_cols']]).fit().summary()

Optimization terminated successfully.
         Current function value: 0.439837
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            Churn_churn   No. Observations:                  996
Model:                          Logit   Df Residuals:                      990
Method:                           MLE   Df Model:                            5
Date:                Wed, 10 Jul 2019   Pseudo R-squ.:                  0.3011
Time:                        10:10:45   Log-Likelihood:                -438.08
converged:                       True   LL-Null:                       -626.81
                                        LLR p-value:                 2.130e-79
==============================================================================================
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Age                            1.6106      0.350      4.603      0.000       0.925       2.296
LastTransaction                1.2837      0.442      2.904      0.004       0.417       2.150
Gender_male                   -2.7721      0.192    -14.409      0.000      -3.149      -2.395
Payment Method_cash           -0.3271      0.323     -1.012      0.311      -0.960       0.306
Payment Method_cheque         -0.2149      0.418     -0.514      0.607      -1.035       0.605
Payment Method_credit card    -1.1688      0.256     -4.573      0.000      -1.670      -0.668
==============================================================================================
"""

<font color = blue>Removing some columns because of high P value and making a new train col list.

In [105]:
col_dict['train_cols_opt'] = ['Age','LastTransaction','Gender_male','Payment Method_credit card']
Logit(df[col_dict['pred_cols']],df[col_dict['train_cols_opt']]).fit().summary()

Optimization terminated successfully.
         Current function value: 0.440357
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            Churn_churn   No. Observations:                  996
Model:                          Logit   Df Residuals:                      992
Method:                           MLE   Df Model:                            3
Date:                Wed, 10 Jul 2019   Pseudo R-squ.:                  0.3003
Time:                        10:13:23   Log-Likelihood:                -438.60
converged:                       True   LL-Null:                       -626.81
                                        LLR p-value:                 2.823e-81
==============================================================================================
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Age                            1.5394      0.343      4.483      0.000       0.866       2.212
LastTransaction                0.9710      0.294      3.303      0.001       0.395       1.547
Gender_male                   -2.8326      0.183    -15.510      0.000      -3.191      -2.475
Payment Method_credit card    -0.9711      0.150     -6.462      0.000      -1.266      -0.677
==============================================================================================
"""

In [158]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2, random_state=0)

clf = LogisticRegression()
clf.fit(df_train[col_dict['train_cols_opt']],df_train[col_dict['pred_cols']])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [159]:
from sklearn.metrics import accuracy_score
print("Accuracy Score on Unseen data for final model is",accuracy_score(df_test[col_dict['pred_cols']],clf.predict(df_test[col_dict['train_cols_opt']])))

Accuracy Score on Unseen data for final model is 0.79


<h2>Task 3</h2>
<br><font color=blue>Calculate the actual odds and probabilities from the coefficients calculated from the logistic model.
    <br><br>Here we will define a function to convert cofficients to Odds and then to Probability. Each os these cofficients are for variables (in sequence) "Age, Last Transaction, Gender being male, Payment Method Credit Card

In [160]:
def logit2oddprob(lst):
    for num,i in enumerate(list(lst)):
        print('Odds from cofficients are',np.exp(i))
        print('Probability are cofficient is',(np.exp(i)/1+np.exp(i)))

In [161]:
col_dict['train_cols_opt']

['Age', 'LastTransaction', 'Gender_male', 'Payment Method_credit card']

In [162]:
logit2oddprob(clf.coef_)

Odds from cofficients are [3.97567986 3.72417245 0.07271991 0.39384601]
Probability are cofficient is [7.95135971 7.44834491 0.14543982 0.78769202]
